In [ ]:
import numpy as np, os

# Util to add a legend and grid to a plot, and resize it

In [ ]:
import matplotlib.pyplot as plt

def plot_util(_plt, height_inch=5):
    _plt.legend()
    _plt.grid()
    fig = _plt.gcf()
    fig.set_size_inches(18.5, height_inch, forward=True)
    _plt.show()

# Util to overlay data from `output_dict` onto an image

In [ ]:
import PIL.Image, PIL.ImageDraw, PIL.ImageFont

def display_image(output_dict, class_of_interest, frame, labels, capture_screenshots=False):
    # Print the results (each image/frame may have multiple objects)
    for i in range(output_dict['num_detections']):
        
        if class_of_interest =='all' or (output_dict.get('detection_classes_' + str(i)) == class_of_interest):
            
            # Extract top-left & bottom-right coordinates of detected objects
            (y1, x1) = output_dict.get('detection_boxes_' + str(i))[0]
            (y2, x2) = output_dict.get('detection_boxes_' + str(i))[1]

            # Prep string to overlay on the image
            display_str = (labels[output_dict.get('detection_classes_%i' % i)]
                           + ': %s%%' % output_dict.get('detection_scores_%i' % i))

            # Overlay bounding boxes, detection class and scores
            frame = draw_bounding_box( 
                        y1, x1, y2, x2,
                        frame, display_str=display_str)

    if capture_screenshots:
        img = PIL.Image.fromarray(frame)
        img.save('captures/photo_%s.jpg' % cur_time)

    # If a display is available, show image on which inference was performed
    if 'DISPLAY' in os.environ:
        img.show()
    
    return frame

# ****************************************************************************
# Copyright(c) 2017 Intel Corporation. 
# License: MIT See LICENSE file in root directory.
# ****************************************************************************

# Utilities to help visualize the output from
# Intel® Movidius™ Neural Compute Stick (NCS)
def draw_bounding_box(y1, x1, y2, x2, 
                      img, 
                      thickness=4, 
                      color=(255, 255, 0),
                      display_str=()):
    """ draw a bounding box on an image to help visualise the nn output
    
    Inputs
        (x1, y1)  = Top left corner of the bounding box
        (x2, y2)  = Bottom right corner of the bounding box
        img       = Image/frame represented as numpy array
        thickness = Thickness of the bounding box's outline
        color     = Color of the bounding box's outline
    """
    img = PIL.Image.fromarray(img)
    draw = PIL.ImageDraw.Draw(img)

    for x in range(0, thickness):
        draw.rectangle([(x1-x, y1-x), (x2-x, y2-x)], outline=color)

    font = PIL.ImageFont.load_default()
    draw.text((x1, y1), display_str, font=font)

    return np.array(img)

# Experiment logger to keep track of EKF afterwards

In [ ]:
class ExperimentLogger():
    def __init__(self):
        self.phi_yaw_arr = []
        self.EKF_yaw_arr = []
        self.gc_yaw_arr = []

        self.phi_pitch_arr = []
        self.EKF_pitch_arr = []
        self.gc_pitch_arr = []

        self.time_arr = []
    
    def log(self, gc_angles, phi_yaw, EKF_yaw, phi_pitch, EKF_pitch, t):
        self.phi_yaw_arr.append(phi_yaw)
        self.EKF_yaw_arr.append(EKF_yaw.get_pos())

        self.phi_pitch_arr.append(phi_pitch)
        self.EKF_pitch_arr.append(EKF_pitch.get_pos())

        self.gc_yaw_arr.append(gc_angles['yaw'])
        self.gc_pitch_arr.append(gc_angles['pitch'])

        self.time_arr.append(t)
    
    def plot():
        plt.subplot(211)
        plt.plot(self.time_arr, self.EKF_yaw_arr, label='EKF estimate of yaw [deg]')
        plt.plot(self.time_arr, self.phi_yaw_arr, label='Raw NN estimate of yaw [deg]')
        plt.plot(self.time_arr, self.gc_yaw_arr, label='Gimbal yaw [deg]')
        plot_util(plt)

        plt.subplot(212)
        plt.plot(self.time_arr, self.EKF_pitch_arr, label='EKF estimate of pitch [deg]')
        plt.plot(self.time_arr, self.phi_pitch_arr, label='Raw NN estimate of pitch [deg]')
        plt.plot(self.time_arr, self.gc_pitch_arr, label='Gimbal pitch [deg]')
        plot_util(plt)

        loop_times_ms = [(t-t_)*1e3 for t_,t in zip(self.time_arr[0:-1], self.time_arr[1:])]
        plt.stem(self.time_arr[:-1], loop_times_ms, label='loop times [ms]')
        plot_util(plt)

# Rolling buffer for gimbal angles

In [1]:
class GimbalAngleBuffer():
    """ Keeps a rolling buffer of the last `buffer_len` angle readings from the gimbal,
        including the time at which the angle readings were taken.
        
        If given the time at which a photo was taken, it returns the gimbal angle log
        which was taken close to that time. """
    def __init__(self, buffer_len=20):
        self.buffer_len = buffer_len
        self.time_buffer = np.zeros(buffer_len)
        self.yaw_buffer = np.zeros(buffer_len)
        self.pitch_buffer = np.zeros(buffer_len)
        self.idx = 0
    
    def log(self, gc_angles):
        self.time_buffer[self.idx % self.buffer_len] = time.time()
        self.yaw_buffer[self.idx % self.buffer_len] = gc_angles['yaw']
        self.pitch_buffer[self.idx % self.buffer_len] = gc_angles['pitch']
        self.idx += 1

    def angle_closest_to(self, photo_time):
        idx = np.argmin(np.abs(self.time_buffer - photo_time)) # time closest to the photo time
        return self.yaw_buffer[idx], self.pitch_buffer[idx]

In [10]:
# test:
# x = GimbalAngleBuffer(15)

# gc_angles1 = {'yaw': 20, 'pitch': 15}
# gc_angles2 = {'yaw': 30, 'pitch': 25}
# gc_angles3 = {'yaw': 40, 'pitch': 35}
# gc_angles4 = {'yaw': 50, 'pitch': 45}

# x.log(gc_angles1)
# time.sleep(0.5)

# x.log(gc_angles2)
# time.sleep(0.2)
# photo_time = time.time()
# time.sleep(0.5)

# x.log(gc_angles3)
# time.sleep(0.5)

# x.log(gc_angles4)
# time.sleep(0.5)

# x.angle_closest_to(photo_time) # should be (30, 25)

(30.0, 25.0)